In [1]:
import numpy as np
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.select import Select
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import StaleElementReferenceException
from selenium import webdriver
import selenium
import time



chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',options=chrome_options)



In [2]:
"""Scraping Algorithm"""

def get_df(url_list):
    title_list = []
    author_list = []
    pub_date_list = []
    abstract_list = []
    link_list = []
    df_affiliations_list = []
    journal_list = []
    publisher_list = []
    keyword_list = []

    for url in url_list:
        r = requests.get(url)
        soup = BeautifulSoup(r.content, 'html.parser')

        '''Scraping Title'''
        title = soup.find("title")
        if title is not None:
            title_list += [title.text]
        else:
            title_list += ['']

        '''Scraping Authors'''
        authors = soup.findAll("span", itemprop="author")  # creates list of all authors with html tags
        all_authors = []
        for author in authors:
            all_authors.append(author.text)
        author_list += [all_authors]

        '''Scraping Publication Date'''
        publication_date = soup.find("span", itemprop="datePublished")
        if publication_date is not None:
            pub_date_list += [publication_date.text]
        else:
            pub_date_list += ['']

        '''Scraping Abstract'''
        abst = []
        abstract = soup.findAll("div", attrs={"class": "so-d", "itemprop": 'description'})
        for i in abstract:
            abst.append(" ".join(i.text.split()))
        # abstract = soup.find("meta", attrs= {"name": "citation_abstract"})
        # abstract_string = abstract["content"].replace('\n', '')
        if len(abst):
            abstract_list += [abst[0]]
        else:
            abstract_list += ['']

        '''Scraping Journal'''
        journal = soup.find("a", itemprop="journal-url")
        if journal is not None:
            journal_list += [journal.text]
        else:
            journal_list += ['']

        '''Scraping Publisher'''
        publisher = soup.find("a", itemprop="publisher-url")
        if publisher is not None:
            publisher_list += [publisher.text]
        else:
            publisher_list += ['']

        '''Getting DOI'''
        table = soup.findAll("div", attrs={'class': 'so-info-entry'})
        emp = ''
        for row in table:
            sub = re.compile('DOI:\s*(\S+)')
            k = sub.findall(row.text)
            for i in k:
                emp += i
        link_list += [emp]

        '''Getting Keywords'''
        tbl = soup.findAll("div", attrs={'class': "so-article-header-info"})
        for row in tbl:
            sub = re.compile('Keywords:(.*)')
            k = sub.findall(row.text)
        if len(k):
            keyword_list += [k[0]]
        else:
            keyword_list += ['']

        '''Scraping Author Affiliations'''
        affiliations_div = soup.find("div", attrs={'class': 'so-affiliations', 'id': 'info-aff-group'})
        # whole div

        if affiliations_div is not None:
            affiliations_specific = affiliations_div.findAll('div', attrs={
                'class': 'so-info-entry'})  # specific div with affiliations
            affiliations_list = []
            for affiliations in affiliations_specific:
                affiliations_list.append(affiliations.text.strip())

            affiliations_list_nospace = []
            for affiliation in affiliations_list:
                affiliation_text = " ".join(affiliation.split())
                # affiliation_text = affiliation.replace('\n', ' ').replace('\r', '').replace('  ', '')
                affiliations_list_nospace.append(affiliation_text)

            affiliations_clean_list = []
            affiliations_regex = "[A-Z]+.+"
            for affiliations in affiliations_list_nospace:
                affiliation_clean = re.findall(f"{affiliations_regex}", affiliations)
                affiliations_clean_list.append(affiliation_clean)

            affiliations_list_final = []
            for affiliations in affiliations_clean_list:
                affiliations_list_final.append(affiliations[0])

            affliliations_list_final_inc_GRID = []
            for i in affiliations_list_final:
                if 'GRID' in i:
                    k = re.findall(f"grid\S+, (.+)", i)
                    affliliations_list_final_inc_GRID.append(k)
                else:
                    affliliations_list_final_inc_GRID.append(i)

            df_affiliations_list += [affliliations_list_final_inc_GRID]
        else:
            df_affiliations_list += ['']
    d = {'Title': title_list, 'Author(s)': author_list, 'Publication Date': pub_date_list, 'Abstract': abstract_list,
         'DOI': link_list, 'URL': url_list, 'Journal': journal_list, 'Publisher': publisher_list,
         'Keywords': keyword_list, 'Affiliations': df_affiliations_list}
    return pd.DataFrame(data=d)


In [3]:
driver.get("https://www.scienceopen.com/search#('v'~4_'id'~''_'queryType'~1_'context'~null_'kind'~77_'order'~0_'orderLowestFirst'~false_'filters'~!*_'hideOthers'~false)")
time.sleep(20)
articles = driver.find_elements(By.TAG_NAME, "a")
lst = list()

for link in articles:
    href = link.get_attribute("href")
    if href is not None:
        if href.__contains__("document"):
            lst.append(str(href))

filtered = filter(lambda href: href.__contains__("document?"), lst)
filtered_list = list(filtered)

without_duplicates = []
for i in filtered_list:
  if i not in without_duplicates:
    without_duplicates.append(i)

print(without_duplicates)
get_df(without_duplicates)


['https://www.scienceopen.com/document?vid=4e6290c9-0ded-40ed-b858-ba18119863c0', 'https://www.scienceopen.com/document?vid=710e8344-8472-43f4-853b-3cb11fc00863', 'https://www.scienceopen.com/document?vid=7dcc5159-f2e5-4300-a24e-6c84e9b0b6be', 'https://www.scienceopen.com/document?vid=5710aa10-21d5-4208-bdbe-30133996962a', 'https://www.scienceopen.com/document?vid=eeb7a3a1-be72-47bc-9ad7-dae8a42d2b00', 'https://www.scienceopen.com/document?vid=96883796-634b-4c0e-88a1-22e384851cc6', 'https://www.scienceopen.com/document?vid=c567b537-7807-44ce-bd44-b0a1effd0352', 'https://www.scienceopen.com/document?vid=269035fa-9d4b-4952-89ee-5b39915c2a4a', 'https://www.scienceopen.com/document?vid=d1a3f531-f1a3-4af9-a37b-055c084931b1', 'https://www.scienceopen.com/document?vid=66017ad2-6b51-4711-9d4a-7a8e6b4a3c42']


,Title,Author(s),Publication Date,Abstract,DOI,URL,Journal,Publisher,Keywords,Affiliations
0,Report 9: Impact of non-pharmaceutical interve...,"[N Ferguson, D Laydon, G Nedjati Gilani, N Ima...",2020,The global impact of COVID-19 has been profoun...,10.25561/77482,https://www.scienceopen.com/document?vid=4e629...,,Imperial College London,"COVID19, Coronavirus, Non-pharmaceutical inte...",[Imperial College London]
1,Covid-19: Researcher blows the whistle on data...,[Paul D Thacker],November 02 2021,,10.1136/bmj.n2635,https://www.scienceopen.com/document?vid=710e8...,BMJ,BMJ,,
2,Chloroquine is a potent inhibitor of SARS coro...,"[Martin J Vincent, Eric Bergeron, Suzanne Benj...",22 August 2005,Background Severe acute respiratory syndrome (...,10.1186/1743-422X-2-69,https://www.scienceopen.com/document?vid=7dcc5...,Virology Journal,BioMed Central,severe acute respiratory syndrome coronavirus...,"[Special Pathogens Brach, Division of Viral an..."
3,Chloroquine is a potent inhibitor of SARS coro...,"[MJ Vincent, E Bergeron, S. BENJANNET, BR Eric...",2005,,:,https://www.scienceopen.com/document?vid=5710a...,,,,
4,The proximal origin of SARS-CoV-2 – ScienceOpen,"[Kristian G. Andersen, Andrew Rambaut, W. Ian ...",17 March 2020,To the Editor — Since the first reports of nov...,10.1038/s41591-020-0820-9,https://www.scienceopen.com/document?vid=eeb7a...,Nature Medicine,Nature Publishing Group US,"molecular evolution, computational biology an...","[[Department of Immunology and Microbiology, ,..."
5,Comparing SARS-CoV-2 natural immunity to vacci...,"[Sivan Gazit, Roei Shlezinger, Galit Perez, Ro...",August 25 2021,Background Reports of waning vaccine-induced i...,10.1101/2021.08.24.21262415,https://www.scienceopen.com/document?vid=96883...,,medRxiv,,
6,Effectiveness of Adding a Mask Recommendation ...,"[Henning Bundgaard, Johan Skov Bundgaard, Dani...",18 November 2020,Observational evidence suggests that mask wear...,10.7326/M20-6817,https://www.scienceopen.com/document?vid=c567b...,Annals of Internal Medicine,American College of Physicians,,"[The Heart Center, Rigshospitalet, Copenhagen ..."
7,Ivermectin for Prevention and Treatment of COV...,"[Andrew Bryant, Theresa A. Lawrie, Therese Dow...",21 June 2021,Repurposed medicines may have a role against t...,10.1097/MJT.0000000000001402,https://www.scienceopen.com/document?vid=26903...,American Journal of Therapeutics,American Journal of Therapeutics,"ivermectin, prophylaxis, treatment, covid-19,...","[Population Health Sciences Institute, Newcast..."
8,Safety and Efficacy of the BNT162b2 mRNA Covid...,"[Fernando P. Polack, Stephen J. Thomas, Nichol...",10 December 2020,Background Severe acute respiratory syndrome c...,10.1056/NEJMoa2034577,https://www.scienceopen.com/document?vid=d1a3f...,The New England Journal of Medicine,Massachusetts Medical Society,,[From Fundacion INFANT (F.P.P.) and iTrials-Ho...
9,Safety and efficacy of an rAd26 and rAd5 vecto...,"[Denis Y Logunov, Inna V Dolzhikova, Dmitry V ...",2 February 2021,Background A heterologous recombinant adenovir...,10.1016/S0140-6736(21)00234-8,https://www.scienceopen.com/document?vid=66017...,"Lancet (London, England)",Elsevier Ltd.,,[Federal State Budget Institution “National Re...


In [44]:



driver = webdriver.Chrome(ChromeDriverManager().install()) # install Chrome webdriver
url = "https://www.scienceopen.com/search"
options = Options()
driver.get(url)

# click "accept all cookies" button
acceptCookiesButton = driver.find_element(By.XPATH, "/html/body/aside/div/div/div[2]/div[2]/div/div[2]/div[1]/button[1]")
acceptCookiesButton.click()

time.sleep(5)

# click "load more" button function
def clickButton():
  time.sleep(10)
  button = driver.find_element(By.XPATH,"//*[@id=\"id1\"]/div/div/div/div[2]/div/div[7]/div[2]/div/button")
  button.click()



for i in range(3):
  clickButton()


def get_links():
  """Returns a list of URL's"""
  articles = driver.find_elements(By.TAG_NAME, "a")
  lst = list()

  for link in articles:
      href = link.get_attribute("href")
      if href is not None:
          if href.__contains__("document"):
              lst.append(str(href))

  filtered = filter(lambda href: href.__contains__("document?"), lst)
  filtered_list = list(filtered)

  without_duplicates = []
  for i in filtered_list:
    if i not in without_duplicates:
      without_duplicates.append(i)

  return without_duplicates

linkList = get_links()
print(linkList)




/var/folders/rk/pmq93j_n5257m49qgjx_syvr0000gn/T/ipykernel_1184/1758167543.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install()) # install Chrome webdriver


StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=107.0.5304.110)
Stacktrace:
0   chromedriver                        0x0000000100ceaa88 chromedriver + 4123272
1   chromedriver                        0x0000000100c76778 chromedriver + 3647352
2   chromedriver                        0x0000000100938ac4 chromedriver + 248516
3   chromedriver                        0x000000010093bc80 chromedriver + 261248
4   chromedriver                        0x000000010093baa4 chromedriver + 260772
5   chromedriver                        0x000000010093c540 chromedriver + 263488
6   chromedriver                        0x00000001009a1b2c chromedriver + 678700
7   chromedriver                        0x00000001009a1020 chromedriver + 675872
8   chromedriver                        0x0000000100962b10 chromedriver + 420624
9   chromedriver                        0x0000000100963c30 chromedriver + 425008
10  chromedriver         